In [1]:
import os
from matplotlib import pyplot as plt

In [5]:
labels = ['(0, 0)', '(1, 1)', '(1, 3)', '(1, 5)', '(1, 7)', '(1, 9)', 
          '(2, 1)', '(2, 3)', '(2, 5)', '(2, 7)', '(2, 9)',
          '(3, 1)', '(3, 3)', '(3, 5)', '(3, 7)', '(3, 9)']
datas = [(48.96, 36.02, 56.70), 
         (50.14, 37.19, 55.16), (47.59, 39.25, 59.57), (49.40, 38.24, 59.02), (49.35, 37.20, 59.14), (49.97, 38.43, 57.32),
         (49.14, 36.59, 55.66), (46.88, 36.60, 57.08), (49.79, 38.37, 57.78), (49.43, 37.43, 59.06), ()]

TypeError: filter expected 2 arguments, got 1

In [4]:
exp_names

['20_baseline_no_mixup_stop',
 '20_proteacher_l1p10',
 '20_baseline_local',
 '19_proteacher_l3p1',
 '19_proteacher_l2p3',
 '19_proteacher_l3p7',
 '19_proteacher_l2p5',
 '20_baseline_prompt',
 '19_proteacher_l2p9',
 '20_baseline_no_mixup',
 '20_proteacher_l3p10',
 '20_baseline_1',
 '20_baseline_1_no_mixup',
 '20_proteacher_l1p1',
 '20_proteacher_l1p5',
 '20_proteacher_l1p3',
 '20_baseline_prompt_none',
 '19_proteacher_l2p1',
 '20_baseline',
 '20_proteacher_l3p1',
 '20_proteacher_l2p3',
 '19_proteacher_l2p7',
 '19_proteacher_l1p9',
 '19_proteacher_l1p1',
 '20_proteacher_l2p5',
 '19_proteacher_l1p3',
 '19_proteacher_l3p3',
 '20_proteacher_l3p5',
 '20_baseline_prompt_all',
 '20_baseline_prompt_w_u',
 '19_proteacher_l3p9',
 '19_proteacher_l1p7',
 '19_baseline',
 '19_proteacher_l3p5',
 '20_proteacher_l2p1',
 '19_proteacher_l1p5']